# Information
## How to
1. Set the parameters. **UPPERCASE** letters are user input variables
2. Run the reprojection cell


# Requirements

In [5]:
# Standard libraries
import os
import rasterio
import numpy as np

from pathlib import Path
from importlib_resources import files

# Custom modules
from beak.utilities.preparation import impute_data
from beak.utilities.io import create_file_list, save_raster, check_path


# Scaling

**Scaling** all numerical folders within a specified model configuration.<br>
Reads the <code>ROOT_FOLDER</code> and takes the <code>NUMERICAL</code> subfolder within each model configuration.

**User inputs**

In [6]:
BASE_PATH = files("beak.data")
BASE_NAME = "AERORADIOMETRICS-PROCESSED"
BASE_SPATIAL = ["EPSG_4326_RES_0_015"]
BASE_EXTENT = "CLIPPED_USC"

input_folders = []
base_rasters = []

for spatial in BASE_SPATIAL:
    input_folder = BASE_PATH / BASE_NAME / spatial / BASE_EXTENT / "USGS" / "UNIFIED"
    base_raster = BASE_PATH / "BASE_RASTERS" / (spatial + "_" + BASE_EXTENT + ".tif")

    input_folders.append(input_folder)
    base_rasters.append(base_raster)
  

**Run**

In [7]:
for i, input_folder in enumerate(input_folders):
  output_folder = Path(str(input_folder) + "_IMPUTED")
  
  base_raster = rasterio.open(base_rasters[i])
  base_meta = base_raster.meta.copy()
  base_raster = base_raster.read()
  
  file_list = create_file_list(input_folder, recursive=False)
  
  for file in file_list:
    raster = rasterio.open(file)
    array = raster.read()

    # Impute data
    print(f"Imputing data for {file}...")
    imputed_array = np.where(array == raster.nodata, np.nan, array)
    imputed_array = impute_data(imputed_array)
    imputed_array = np.where(base_raster == base_meta["nodata"], np.nan, imputed_array)
    
    out_path = output_folder / Path(file).name
    check_path(Path(os.path.dirname(out_path)))
    
    # Save
    save_raster(out_path, array=imputed_array, dtype="float32", metadata=raster.meta)   


Imputing data for s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\AERORADIOMETRICS-PROCESSED\EPSG_4326_RES_0_015\CLIPPED_USC\USGS\UNIFIED\NAMrad_K.tif...
Imputing data for s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\AERORADIOMETRICS-PROCESSED\EPSG_4326_RES_0_015\CLIPPED_USC\USGS\UNIFIED\NAMrad_Th.tif...
Imputing data for s:\projekte\20230082_darpa_criticalmaas_ta3\bearbeitung\github\beak-ta3\src\beak\data\AERORADIOMETRICS-PROCESSED\EPSG_4326_RES_0_015\CLIPPED_USC\USGS\UNIFIED\NAMrad_U.tif...
